In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

DB Password: ········


In [2]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [3]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [4]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        for mrn in cohort_indexes: 
            x = get_mrn_has_certain_condition_WINDOW(mrn,cohort_condition,df_mrn.loc[mrn]["HT_Onset"]-gap_in_days)
            a = [mrn, x]
            cohort_mrn_diagnosis.append(a)
    else:
        print("function not available")
    #convert list cohort_mrn_diagnosis to panda
    #condition_name= 'has_condition_'+name_feature
    #col_names = ['medical_record_number',condition_name]
    #df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    #df_final.set_index('medical_record_number', inplace=True)
    #df_final=df_mrn.merge( df_final, left_index=True, right_index=True)
    #df_final.to_parquet(name_df)
    #print(df_final)
    df_final = cohort_mrn_diagnosis
    return df_final

In [5]:
condition_cr = (Diagnosis("430%", "ICD-9")| Diagnosis("I60%", "ICD-10")| 
                Diagnosis("I61%", "ICD-10")| Diagnosis("I62%", "ICD-10")|
                Diagnosis("I63%", "ICD-10")| Diagnosis("I64%", "ICD-10")|
                Diagnosis("I65%", "ICD-10")| Diagnosis("431%", "ICD-9") |
                Diagnosis("432%", "ICD-9") | Diagnosis("433%", "ICD-9") |
                Diagnosis("434%", "ICD-9") | Diagnosis("435%", "ICD-9") |
                Diagnosis("436%", "ICD-9") | Diagnosis("437%", "ICD-9") |
                Diagnosis("438%", "ICD-9") | Diagnosis("I66%", "ICD-10")|
                Diagnosis("I67%", "ICD-10")| Diagnosis("I68%", "ICD-10")
                )


In [7]:
cr_COUNT = get_has_certain_condition(condition_cr , Case_filtered_15_540, "cr", 180 , "COUNT", "Case")


Fetching data for Diagnosis (...)


In [8]:
cr_COUNT

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_cr
medical_record_number,,,,,,,,,,,,,
100067499,82,24103.0,NaN,24106.0,24103.0,Age_BP_condition,21049,3054.0,1951,White,Other,Male,0.0
100091035,154,11369.0,12938.0,12881.0,11369.0,Age_BP_condition,9141,2228.0,1978,Ba,Other,Male,0.0
1004105957,43,22961.0,NaN,24104.0,22961.0,Age_BP_condition,19914,3047.0,1952,White,Greek Orthodox,Male,1.0
100423963,20,NaN,23584.0,23584.0,23584.0,Age_ICD_condition,19827,3757.0,1945,Other,Pt Declined,Male,0.0
1005188482,279,20636.0,20514.0,20514.0,20514.0,Age_ICD_condition,19884,630.0,1958,Ba,Jewish,Male,0.0
1006954100,28,15242.0,15536.0,15536.0,15242.0,Age_BP_condition,13828,1414.0,1973,Unknown,Catholic,Female,0.0
1006987597,72,27618.0,27618.0,27618.0,27618.0,Age_BP_condition,26706,912.0,1941,White,Catholic,Male,0.0
1007749798,93,NaN,27233.0,27233.0,27233.0,Age_ICD_condition,25235,1998.0,1939,White,Catholic,Female,0.0
1007977434,221,25355.0,24347.0,24347.0,24347.0,Age_ICD_condition,23605,742.0,1943,White,Unknown,Male,0.0


In [9]:
cr_COUNT.loc[cr_COUNT['number_of_occurences_cr'] != 0]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_cr
medical_record_number,,,,,,,,,,,,,
1004105957,43,22961.0,NaN,24104.0,22961.0,Age_BP_condition,19914,3047.0,1952,White,Greek Orthodox,Male,1.0
111571287,95,31180.0,NaN,31339.0,31180.0,Age_BP_condition,25929,5251.0,1929,White,Pt Declined,Male,1.0
1173163870,402,20441.0,20504.0,20504.0,20441.0,Age_BP_condition,12432,8009.0,1956,Other,Pt Declined,Male,19.0
1179295872,182,27353.0,27353.0,27799.0,27353.0,Age_BP_condition,25551,1802.0,1941,White,Unknown,Male,1.0
1214485155,2354,19820.0,20020.0,19838.0,19820.0,Age_BP_condition,16427,3393.0,1957,White,Catholic,Male,6.0
1428367036,469,28775.0,28775.0,28888.0,28775.0,Age_BP_condition,24963,3812.0,1934,Other,Catholic,Female,2.0
1573256350,183,29383.0,29341.0,NaN,29341.0,Age_ICD_condition,26310,3031.0,1935,White,Catholic,Male,2.0
160536982,92,28798.0,28268.0,28268.0,28268.0,Age_ICD_condition,24724,3544.0,1938,Ba,Baptist,Female,1.0
1672330252,81,NaN,21774.0,21774.0,21774.0,Age_ICD_condition,21099,675.0,1949,White,Jewish,Male,1.0


In [10]:
cr_EVER = get_has_certain_condition(condition_cr, Case_filtered_15_540, "cr", 180 , "EVER", "Case")


Fetching data for Diagnosis (...)


In [11]:
cr_EVER

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_cr
medical_record_number,,,,,,,,,,,,,
100067499,82,24103.0,NaN,24106.0,24103.0,Age_BP_condition,21049,3054.0,1951,White,Other,Male,0.0
100091035,154,11369.0,12938.0,12881.0,11369.0,Age_BP_condition,9141,2228.0,1978,Ba,Other,Male,0.0
1004105957,43,22961.0,NaN,24104.0,22961.0,Age_BP_condition,19914,3047.0,1952,White,Greek Orthodox,Male,1.0
100423963,20,NaN,23584.0,23584.0,23584.0,Age_ICD_condition,19827,3757.0,1945,Other,Pt Declined,Male,0.0
1005188482,279,20636.0,20514.0,20514.0,20514.0,Age_ICD_condition,19884,630.0,1958,Ba,Jewish,Male,0.0
1006954100,28,15242.0,15536.0,15536.0,15242.0,Age_BP_condition,13828,1414.0,1973,Unknown,Catholic,Female,0.0
1006987597,72,27618.0,27618.0,27618.0,27618.0,Age_BP_condition,26706,912.0,1941,White,Catholic,Male,0.0
1007749798,93,NaN,27233.0,27233.0,27233.0,Age_ICD_condition,25235,1998.0,1939,White,Catholic,Female,0.0
1007977434,221,25355.0,24347.0,24347.0,24347.0,Age_ICD_condition,23605,742.0,1943,White,Unknown,Male,0.0


In [12]:
cr_EVER.loc[cr_EVER['has_condition_cr'] == 1]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_cr
medical_record_number,,,,,,,,,,,,,
1004105957,43,22961.0,NaN,24104.0,22961.0,Age_BP_condition,19914,3047.0,1952,White,Greek Orthodox,Male,1.0
111571287,95,31180.0,NaN,31339.0,31180.0,Age_BP_condition,25929,5251.0,1929,White,Pt Declined,Male,1.0
1173163870,402,20441.0,20504.0,20504.0,20441.0,Age_BP_condition,12432,8009.0,1956,Other,Pt Declined,Male,1.0
1179295872,182,27353.0,27353.0,27799.0,27353.0,Age_BP_condition,25551,1802.0,1941,White,Unknown,Male,1.0
1214485155,2354,19820.0,20020.0,19838.0,19820.0,Age_BP_condition,16427,3393.0,1957,White,Catholic,Male,1.0
1428367036,469,28775.0,28775.0,28888.0,28775.0,Age_BP_condition,24963,3812.0,1934,Other,Catholic,Female,1.0
1573256350,183,29383.0,29341.0,NaN,29341.0,Age_ICD_condition,26310,3031.0,1935,White,Catholic,Male,1.0
160536982,92,28798.0,28268.0,28268.0,28268.0,Age_ICD_condition,24724,3544.0,1938,Ba,Baptist,Female,1.0
1672330252,81,NaN,21774.0,21774.0,21774.0,Age_ICD_condition,21099,675.0,1949,White,Jewish,Male,1.0


In [13]:
Cerebrovascular_disease = cr_EVER.merge(cr_COUNT['number_of_occurences_cr'], left_index=True, right_index=True)


In [14]:
Cerebrovascular_disease

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_cr,number_of_occurences_cr
medical_record_number,,,,,,,,,,,,,,
100067499,82,24103.0,NaN,24106.0,24103.0,Age_BP_condition,21049,3054.0,1951,White,Other,Male,0.0,0.0
100091035,154,11369.0,12938.0,12881.0,11369.0,Age_BP_condition,9141,2228.0,1978,Ba,Other,Male,0.0,0.0
1004105957,43,22961.0,NaN,24104.0,22961.0,Age_BP_condition,19914,3047.0,1952,White,Greek Orthodox,Male,1.0,1.0
100423963,20,NaN,23584.0,23584.0,23584.0,Age_ICD_condition,19827,3757.0,1945,Other,Pt Declined,Male,0.0,0.0
1005188482,279,20636.0,20514.0,20514.0,20514.0,Age_ICD_condition,19884,630.0,1958,Ba,Jewish,Male,0.0,0.0
1006954100,28,15242.0,15536.0,15536.0,15242.0,Age_BP_condition,13828,1414.0,1973,Unknown,Catholic,Female,0.0,0.0
1006987597,72,27618.0,27618.0,27618.0,27618.0,Age_BP_condition,26706,912.0,1941,White,Catholic,Male,0.0,0.0
1007749798,93,NaN,27233.0,27233.0,27233.0,Age_ICD_condition,25235,1998.0,1939,White,Catholic,Female,0.0,0.0
1007977434,221,25355.0,24347.0,24347.0,24347.0,Age_ICD_condition,23605,742.0,1943,White,Unknown,Male,0.0,0.0


In [15]:
Cerebrovascular_disease.to_parquet("Cerebrovascular_disease.parquet")

In [ ]:
#### CONTROLS ####

In [6]:
cr_COUNT = get_has_certain_condition(condition_cr , Control_filtered_15_540, "cr", 180 , "COUNT", "Control")

Fetching data for Diagnosis (...)


In [7]:
cr_EVER = get_has_certain_condition(condition_cr, Control_filtered_15_540, "cr", 180 , "EVER", "Control")


Fetching data for Diagnosis (...)


In [10]:
cr_EVER.loc[cr_EVER['has_condition_cr'] == 1]

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_cr
medical_record_number,,,,,,,,,
1000925805,23372,79,21264,2108,1954,White,None,Female,1.0
1003921253,11878,182,6578,5300,1983,Black Or African-American,Baptist,Female,1.0
1007269976,23644,22,20219,3425,1951,White,Jewish,Male,1.0
1012395577,16273,391,11589,4684,1972,Unknown,Christian,Female,1.0
1014652378,20168,61,19009,1159,1963,White,Pt Declined,Male,1.0
1014781566,23819,190,20083,3736,1953,White,Unknown,Female,1.0
1016950881,6570,38,68,6502,2000,White,Jewish,Female,1.0
1017944292,23973,48,22644,1329,1951,White,Unknown,Female,1.0
1022648452,19984,69,15959,4025,1956,White,Catholic,Male,1.0


In [8]:
Cerebrovascular_disease_Control = cr_EVER.merge(cr_COUNT['number_of_occurences_cr'], left_index=True, right_index=True)


In [9]:
Cerebrovascular_disease_Control

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,has_condition_cr,number_of_occurences_cr
medical_record_number,,,,,,,,,,
100002558,21414,15,19951,1463,1959,White,Other,Female,0.0,0.0
100002884,23929,44,18526,5403,1952,Unknown,,Male,0.0,0.0
1000068212,10116,29,9228,888,1990,White,Unknown,Female,0.0,0.0
1000083464,25630,20,23026,2604,1948,White,Catholic,Male,0.0,0.0
1000119948,17087,163,15757,1330,1971,African-American,None,Male,0.0,0.0
1000165222,17506,627,12215,5291,1970,Other,Catholic,Female,0.0,0.0
1000199143,26941,141,22805,4136,1944,White,Jewish,Female,0.0,0.0
1000205924,14867,108,10618,4249,1976,Unknown,Unknown,Female,0.0,0.0
1000212584,19220,15,16818,2402,1965,White,,Male,0.0,0.0


In [11]:
Cerebrovascular_disease_Control.to_parquet("Cerebrovascular_disease_Control.parquet")